# Part 1: Basic Pathfinding

## Depth First Search

In [3]:
#load maze from .lay file
def loadMaze(filename):
	file = open(filename, 'r')
	a = 0
	maze = []
	start = []
	goal = []
	for line in file:
		b = 0
		maze.append([])
		for element in line:
			if element != '\n':
				maze[a].append(element)
				if element == 'P':
					start.append(a)
					start.append(b)
				if element == '.':
					goal.append(a)
					goal.append(b)
			b+=1
		a+=1
	
	file.close()
	return maze, start, goal

#display maze
def printMaze(maze):
	print('\n'.join(map(''.join, maze)))


# depth first search method
def dfs(filename):
	maze, start, goal = loadMaze(filename)
	if (start == goal):
		return True
	finish = False
	nodes = []
	path = []
	visited = []
	frontier = [start]
	maxdepth =0
	fringe = 0
	while (finish == False and len(frontier) > 0):
		# if goal state is reached exit loop
		if (frontier[len(frontier)-1] == goal):
			path.append(frontier[len(frontier)-1])
			maxdepth = max(maxdepth, len(path)-1)
			finish = True
			break
		node = frontier.pop()
		nodes.append(node)
		path.append(node)
		maxdepth = max(maxdepth, len(path)-1)
		if (node not in visited):
			visited.append(node) 
		neighbors = neighborNodes(node, maze, visited)
		if (len(neighbors) > 0):
			for neighbor in neighbors:
				frontier.append(neighbor)
				fringe = max(fringe, len(frontier))
		else:
			while(len(neighborNodes(path[len(path)-1], maze, visited)) == 0):
				path.pop()

	if (finish == True):
		for point in path:
			maze[point[0]][point[1]] = '.'
		#print solution
		print('Solution:')
		printMaze(maze)
		print('Path cost: %d' %(len(path)))
		print('Number of nodes expanded: %d' %(len(nodes)))
		print('Maximum tree depth searched: %d' %(maxdepth))
		print('Maximum size of fringe: %d' %(fringe))

#find neighboring nodes
def neighborNodes(node, maze, visited):
    directions = [
        (1, 0),   # down
        (0, -1),  # left
        (-1, 0),  # up
        (0, 1)    # right
    ]
    
    neighbors = []
    
    for direction in directions:
        newNode = [node[0] + direction[0], node[1] + direction[1]]
        if (0 <= newNode[0] < len(maze) and 
            0 <= newNode[1] < len(maze[0]) and 
            newNode not in visited and 
            maze[newNode[0]][newNode[1]] != '%'):
            neighbors.append(newNode)
    
    return neighbors

#Perfrom depth first search on 4 maze files
for mazefile in ['smallMaze.lay', 'mediumMaze.lay', 'bigMaze.lay', 'openMaze.lay']:
	dfs(mazefile)

Solution:
%%%%%%%%%%%%%%%%%%%%%%
% %%        % %      %
%    %%%%%% % %%%%%% %
%%%%%%     ...%      %
%    % %%%%%%.%% %%%%%
% %%%% %     ....%...%
%        %%% %%%...%.%
%%%%%%%%%%....%%%%%%.%
%..........%%........%
%%%%%%%%%%%%%%%%%%%%%%
Number of nodes expanded: 79
Maximum tree depth searched: 44
Maximum size of fringe: 6
Solution:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%..................................%
%.%%%%%%%%%%%%%%%%%%%%%%% %%%%%%%% %
%.%%...%...%      %%%%%%%   %%     %
%.%%.%.%.%.% %%%% %%%%%%%%% %% %%%%%
%.%%.%.%.%.%.........    %% %%     %
%.%%.%.%.%.%.% %%%% .%%%    %%%%%% %
%.% .%.%.%...%    %%.%%%%%%%%      %
%.%%.%.%.%%%%%%%% %%........%% %%%%%
%.%%.%...%%       %%%%%%%%%.%%     %
%....%%%%%% %%%%%%%      %%.%%%%%% %
%%%%%%      %       %%%% %%.%      %
%      %%%%%% %%%%% %    %%.%% %%%%%
% %%%%%%      %       %%%%%.%%     %
%        %%%%%% %%%%%%%%%%%.%%  %% %
%%%%%%%%%%..................%%%%%% %
%..........%%%%%%%%%%%%%%%%        %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

## A Star Search

In [6]:
import heapq

# Define possible movements (up, down, left, right)
movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

# Define node class
class Node:
    def __init__(self, position, cost, heuristic):
        """
        Initialize a Node.

        Args:
        position: The node's position as a tuple of two integers.
        cost: The cost to reach this node.
        heuristic: The heuristic value for this node.
        """
        self.position = position
        self.cost = cost
        self.heuristic = heuristic
        self.parent = None  # Track the path

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

def heuristic(node, goal):
    """
    Calculate the Manhattan distance heuristic.

    Args:
    node: The current node's position.
    goal: The goal position.

    Returns:
    The Manhattan distance between the node and the goal.
    """
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

def astar_search(grid, start, goal):
    """
    Perform A* search on the grid.

    Args:
    grid: The maze grid as a list of strings.
    start: The start position.
    goal: The goal position.

    Returns:
    A tuple containing the path, path cost, nodes expanded, max tree depth, and max fringe size, or None if no path is found.
    """
    open_set = []
    closed_set = set()
    came_from = {start: None}
    cost_so_far = {start: 0}
    max_tree_depth = 0
    max_fringe_size = 0

    start_node = Node(start, 0, heuristic(start, goal))
    heapq.heappush(open_set, start_node)

    while open_set:
        current_node = heapq.heappop(open_set)
        max_tree_depth = max(max_tree_depth, current_node.cost)

        if current_node.position == goal:
            # Path found, reconstruct and return it
            path = []
            current = current_node
            while current:
                path.insert(0, current.position)
                current = current.parent
            path_cost = len(path) - 1
            nodes_expanded = len(came_from)
            return path, path_cost, nodes_expanded, max_tree_depth, max_fringe_size

        closed_set.add(current_node.position)

        for movement in movements:
            neighbor = (current_node.position[0] + movement[0], current_node.position[1] + movement[1])
            if (0 <= neighbor[0] < len(grid) and 
                0 <= neighbor[1] < len(grid[0]) and 
                grid[neighbor[0]][neighbor[1]] != '%'):
                
                new_cost = current_node.cost + 1
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    heuristic_val = heuristic(neighbor, goal)
                    new_node = Node(neighbor, new_cost, heuristic_val)
                    new_node.parent = current_node
                    came_from[neighbor] = current_node

                    # Check if the neighbor is already in the open set with a lower cost
                    existing_node = next((node for node in open_set if node.position == neighbor), None)
                    if existing_node and existing_node.cost <= new_cost:
                        continue

                    heapq.heappush(open_set, new_node)
                    max_fringe_size = max(max_fringe_size, len(open_set))

    return None  # No path found


def load_maze(filename):
    maze = []
    start = None
    goal = None
    with open(filename, 'r') as f:
        for line in f:
            row = []
            for char in line.strip():
                if char == 'P':
                    start = (len(maze), len(row))
                elif char == '.':
                    goal = (len(maze), len(row))
                row.append(char)
            maze.append(row)
    return maze, start, goal

def visualize_solution(maze, path):
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if (i, j) in path:
                print('.', end='')
            else:
                print(maze[i][j], end='')
        print()

# Run A* search on each maze
for maze_file in ['smallMaze.lay', 'mediumMaze.lay', 'bigMaze.lay', 'openMaze.lay']:
    maze, start, goal = load_maze(maze_file)
    path, path_cost, nodes_expanded, max_tree_depth, max_fringe_size = astar_search(maze, start, goal)
    visualize_solution(maze, path)
    print(maze_file)
    print(f"Path Cost: {path_cost}")
    print(f"Nodes Expanded: {nodes_expanded}")
    print(f"Max Tree Depth: {max_tree_depth}")
    print(f"Max Fringe Size: {max_fringe_size}")

%%%%%%%%%%%%%%%%%%%%%%
% %%        % %      %
%    %%%%%% % %%%%%% %
%%%%%%     ...%      %
%    % %%%%%%.%% %%%%%
% %%%% %    ..   %   %
%        %%%.%%%   % %
%%%%%%%%%%... %%%%%% %
%..........%%        %
%%%%%%%%%%%%%%%%%%%%%%
smallMaze.lay
Path Cost: 19
Nodes Expanded: 59
Max Tree Depth: 19
Max Fringe Size: 8
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                        ..........%
% %%%%%%%%%%%%%%%%%%%%%%%.%%%%%%%% %
% %%   %   %      %%%%%%%...%%     %
% %% % % % % %%%% %%%%%%%%%.%% %%%%%
% %% % % % %        .....%%.%%     %
% %% % % % % % %%%% .%%%....%%%%%% %
% %  % % %   %    %%.%%%%%%%%      %
% %% % % %%%%%%%% %%........%% %%%%%
% %% %   %%       %%%%%%%%%.%%     %
%    %%%%%% %%%%%%%      %%.%%%%%% %
%%%%%%      %       %%%% %%.%      %
%      %%%%%% %%%%% %    %%.%% %%%%%
% %%%%%%      %       %%%%%.%%     %
%        %%%%%% %%%%%%%%%%%.%%  %% %
%%%%%%%%%%..................%%%%%% %
%..........%%%%%%%%%%%%%%%%        %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
mediumMaze.lay
Path

# Part 2: Search with Multiple Goals

## Depth First Search

In [10]:
# Load maze and find all goals in the maze

def loadMazeWithGoals(filename):     #defines the function to load the maze from a .lay file 
    file = open(filename, 'r')       #opens the specified file 
    a = 0                            #initializes variables to track the row number (a)
    maze = []                        #the maze as a 2D list 
    start = []                       #Pacman's starting position 
    goals = []                       #goal positions 


    #following chunk of code reads a maze from a file and keeps track of where Pacman and the goals (marked as dots .) are located.
    
    for line in file:                #iterates through each line 
        b = 0                        #column index
        maze.append([])              #creates a new row in the maze (eventually become a 2D grid where each element of the list corresponds to a character in the maze)     
        for element in line:         #iterates through each element in line
            if element != '\n':      #ignore new line characters
                maze[a].append(element)   
                if element == 'P':    #when starting position (P) is found, the start list is updated 
                    start = [a, b]  
                if element == '.':    #when a goal (.) is found, its cordinates are added to goals list
                    goals.append([a, b]) 
            b += 1
        a += 1
    
    file.close()  
    return maze, start, goals 

# displaying the maze for visualization(optional?)
def printMaze(maze):
    from IPython.display import display, Markdown
    maze_str = '\n'.join(map(''.join, maze))
    display(Markdown(f"```{maze_str}```"))

maze, start, goals = loadMazeWithGoals('smallMaze.lay')
printMaze(maze)

#Helper Function to Find Neighbors
# function finds all valid moves Pacman can make from its current position
#and it returns a list of neighboring cells that are open (e.g  not walls) and haven’t been visited yet

def neighborNodes(node, maze, visited):
    directions = [            #defining of directions teh Pacman can move
        (1, 0),   # down
        (0, -1),  # left
        (-1, 0),  # up
        (0, 1)    # right
    ]
    
    neighbors = []  # this list will store all valid neighboring nodes that Pacman can move to

    #This loop iterates through each of the four possible directions 
    #and calculates coordinates (newNode) of the neighboring cell by adding the direction to the current node's coordinates
    for direction in directions:
        newNode = [node[0] + direction[0], node[1] + direction[1]]
        if (0 <= newNode[0] < len(maze) and 
            0 <= newNode[1] < len(maze[0]) and 
            newNode not in visited and 
            maze[newNode[0]][newNode[1]] != '%'):
            neighbors.append(newNode)
    return neighbors

#In Part 1, Pacman was only required to find a path from the startto  a single goal
#In Part 2, the problem becomes more complex because there are multiple goals, and Pacman must "eat all dots"

#Depth First Search for multiple goals
def dfs_multiple_goals(filename):
   
    maze, start, all_goals = loadMazeWithGoals(filename)
    
    remaining_goals = all_goals.copy()  #holds a copy of all the goals that Pacman hasn't reached yet
                                        #the search continues until remaining_goals is empty, meaning all goals have been visited
    current_position = start
    total_path = []                                      #initializing tracking variables
    total_nodes_expanded = 0
    total_max_depth = 0
    total_fringe_size = 0
    
    #perform DFS for each goal
    while remaining_goals:   #loop ensures that DFS is performed repeatedly, each time targeting the next goal
        print(f"Current position: {current_position}, Remaining goals: {len(remaining_goals)}")  
        finish = False   #flag to indicate whether the current goal has been reached
        nodes = []     #tracking nodes expanded during this DFS run
        path = []   # a list to store the path from Pacman's current position to the current goal
        visited = [] #let us know which nodes have already been visited
        frontier = [current_position]   #stack that stores the nodes to be explored, initialized with Pacman's current position
        max_depth = 0
        fringe = 0
        current_goal = remaining_goals[0]

        #main DFS loop
        while (not finish and frontier):
            # if goal state is reached, exit the loop
            if frontier[-1] == current_goal:
                path.append(frontier[-1])
                max_depth = max(max_depth, len(path)-1)
                finish = True
                break
            
            node = frontier.pop()  #removes the current node from the stack 
            nodes.append(node)     #keep track of all expanded nodes
            path.append(node)      #adds the node to the path being taken towards the goal.
            max_depth = max(max_depth, len(path)-1)  #updates if the current depth is greater than the previous
            
            if node not in visited:         #checks if the node has been visited, if no -  it adds it to the visited list
                visited.append(node)
            
            neighbors = neighborNodes(node, maze, visited)  #DFS explores each node by looking at its neighbors, moving deeply down one path before backtracking to explore others
            if neighbors:
                for neighbor in neighbors:
                    frontier.append(neighbor)
                    fringe = max(fringe, len(frontier))  #tracking fringe size helps analyze fficiency and memory use.
            else:                                              #if there are no neighbors to explore, DFS backtracks by removing nodes from the path
                while len(neighborNodes(path[-1], maze, visited)) == 0:
                    path.pop()
        
        # update the maze with the path found
        for point in path:
            maze[point[0]][point[1]] = '.'
        total_path.extend(path)
        
        # current goal as reached
        remaining_goals.remove(current_goal)
        current_position = current_goal
        
        # update statistics
        total_nodes_expanded += len(nodes)
        total_max_depth = max(total_max_depth, max_depth)
        total_fringe_size = max(total_fringe_size, fringe)
    
    # print final solution
    print('Final Solution Path:')
    printMaze(maze)
    print('Total nodes expanded: %d' % total_nodes_expanded)
    print('Maximum tree depth searched: %d' % total_max_depth)
    print('Maximum size of fringe: %d' % total_fringe_size)

# Run the DFS algorithm on the specified mazes for Part 2
for mazefile in ['tinySearch.lay', 'smallSearch.lay', 'trickySearch.lay']:
    print(f"Running DFS on {mazefile}...")
    dfs_multiple_goals(mazefile)
    print("\n\n")

```%%%%%%%%%%%%%%%%%%%%%%
% %%        % %      %
%    %%%%%% % %%%%%% %
%%%%%%     P  %      %
%    % %%%%%% %% %%%%%
% %%%% %         %   %
%        %%% %%%   % %
%%%%%%%%%%    %%%%%% %
%.         %%        %
%%%%%%%%%%%%%%%%%%%%%%```

Running DFS on tinySearch.lay...
Current position: [3, 4], Remaining goals: 10
Current position: [1, 1], Remaining goals: 9
Current position: [1, 2], Remaining goals: 8
Current position: [1, 6], Remaining goals: 7
Current position: [1, 7], Remaining goals: 6
Current position: [2, 4], Remaining goals: 5
Current position: [4, 1], Remaining goals: 4
Current position: [4, 7], Remaining goals: 3
Current position: [5, 1], Remaining goals: 2
Current position: [5, 3], Remaining goals: 1
Final Solution Path:


```%%%%%%%%%
%.......%
%%%%.%%.%
%.......%
%.%% %%.%
%.%.....%
%%%%%%%%%```

Total nodes expanded: 100
Maximum tree depth searched: 16
Maximum size of fringe: 6



Running DFS on smallSearch.lay...
Current position: [1, 16], Remaining goals: 17
Current position: [1, 1], Remaining goals: 16
Current position: [1, 13], Remaining goals: 15
Current position: [1, 14], Remaining goals: 14
Current position: [1, 15], Remaining goals: 13
Current position: [1, 18], Remaining goals: 12
Current position: [2, 1], Remaining goals: 11
Current position: [2, 4], Remaining goals: 10
Current position: [2, 7], Remaining goals: 9
Current position: [2, 10], Remaining goals: 8
Current position: [2, 13], Remaining goals: 7
Current position: [2, 18], Remaining goals: 6
Current position: [3, 6], Remaining goals: 5
Current position: [3, 7], Remaining goals: 4
Current position: [3, 8], Remaining goals: 3
Current position: [3, 9], Remaining goals: 2
Current position: [3, 10], Remaining goals: 1
Final Solution Path:


```%%%%%%%%%%%%%%%%%%%%
%..................%
%.%%.%%.%%.%%.%%.%.%
% %% %...........%.%
%%%%%%%%%%%%%%%%%%%%```

Total nodes expanded: 225
Maximum tree depth searched: 28
Maximum size of fringe: 8



Running DFS on trickySearch.lay...
Current position: [3, 9], Remaining goals: 13
Current position: [1, 1], Remaining goals: 12
Current position: [1, 13], Remaining goals: 11
Current position: [1, 14], Remaining goals: 10
Current position: [2, 1], Remaining goals: 9
Current position: [2, 4], Remaining goals: 8
Current position: [2, 7], Remaining goals: 7
Current position: [2, 10], Remaining goals: 6
Current position: [2, 13], Remaining goals: 5
Current position: [5, 1], Remaining goals: 4
Current position: [5, 2], Remaining goals: 3
Current position: [5, 3], Remaining goals: 2
Current position: [5, 4], Remaining goals: 1
Final Solution Path:


```%%%%%%%%%%%%%%%%%%%%
%..............%...%
%.%%.%%.%%.%%.%%.%.%
%................%.%
%%%%%%%%%%%%%%%%%%.%
%..................%
%%%%%%%%%%%%%%%%%%%%```

Total nodes expanded: 287
Maximum tree depth searched: 55
Maximum size of fringe: 8





## A Star Search

In [13]:
import heapq

class SearchNode:
    def __init__(self, position, predecessor=None, cost_from_start=0, heuristic_value=0):
        self.position = position
        self.predecessor = predecessor
        self.cost_from_start = cost_from_start  # cost incurred from the start to the current node
        self.heuristic_value = heuristic_value  # heuristic value (Manhattan distance to closest unvisited goal)

    def total_cost(self):
        return self.cost_from_start + self.heuristic_value

    def __lt__(self, other):
        return self.total_cost() < other.total_cost()

def calculate_manhattan_distance(current_position, goal_positions):
    # Calculate Manhattan distance to the closest unvisited goal
    min_distance = float('inf')
    for goal in goal_positions:
        distance = abs(current_position[0] - goal[0]) + abs(current_position[1] - goal[1])
        min_distance = min(min_distance, distance)
    return min_distance

def find_adjacent_positions(current_position, maze_grid):
    adjacent_positions = []
    for move in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        new_position = (current_position[0] + move[0], current_position[1] + move[1])
        if 0 <= new_position[0] < len(maze_grid) and 0 <= new_position[1] < len(maze_grid[0]) and maze_grid[new_position[0]][new_position[1]] != '%':
            adjacent_positions.append(new_position)
    return adjacent_positions

def perform_astar_search(start_position, goal_positions, maze_grid):
    open_list = []
    heapq.heappush(open_list, SearchNode(start_position, None, 0, calculate_manhattan_distance(start_position, goal_positions)))

    visited_positions = set()
    nodes_expanded = 0
    max_search_depth = 0
    max_open_list_size = 1

    while open_list:
        current_node = heapq.heappop(open_list)
        if current_node.position in goal_positions:
            goal_positions.remove(current_node.position)
            if not goal_positions:  # All goals reached
                path = []
                while current_node:
                    path.append(current_node.position)
                    current_node = current_node.predecessor
                path.reverse()
                return path, nodes_expanded, max_search_depth, max_open_list_size

        if current_node.position not in visited_positions:
            visited_positions.add(current_node.position)
            for adjacent_position in find_adjacent_positions(current_node.position, maze_grid):
                cost_from_start = current_node.cost_from_start + 1
                heuristic_value = calculate_manhattan_distance(adjacent_position, goal_positions)
                new_node = SearchNode(adjacent_position, current_node, cost_from_start, heuristic_value)
                heapq.heappush(open_list, new_node)
                max_open_list_size = max(max_open_list_size, len(open_list))
            nodes_expanded += 1
            max_search_depth = max(max_search_depth, current_node.cost_from_start)

    return None, None, None, None

def display_solution(maze_grid, solution_path):
    for i in range(len(maze_grid)):
        for j in range(len(maze_grid[0])):
            if (i, j) in solution_path:
                print('.', end=' ')
            else:
                print(maze_grid[i][j], end=' ')
        print()

def solve_maze_problem(maze_grid):
    goal_positions = []
    for i in range(len(maze_grid)):
        for j in range(len(maze_grid[0])):
            if maze_grid[i][j] == 'P':
                start_position = (i, j)
            elif maze_grid[i][j] == '.':
                goal_positions.append((i, j))
    if not goal_positions:
        print("No goals found")
        return

    solution_path, nodes_expanded, max_search_depth, max_open_list_size = perform_astar_search(start_position, goal_positions, maze_grid)
    if solution_path:
        print("Solution:")
        display_solution(maze_grid, solution_path)
        print("Path cost:", len(solution_path) - 1)
        print("Number of nodes expanded:", nodes_expanded)
        print("Max tree depth:", max_search_depth)
        print("Max fringe size:", max_open_list_size)
    else:
        print("No solution found")

def load_maze_from_file(file_name):
    maze_grid = []
    with open(file_name, 'r') as file:
        for line in file:
            maze_grid.append(line.strip())
    return maze_grid

# Loop through each maze file and solve the maze
for maze_file in ['tinySearch.lay', 'smallSearch.lay', 'trickySearch.lay']:
    maze_grid = load_maze_from_file(maze_file)
    print(f"Solving maze from file: {maze_file}")
    solve_maze_problem(maze_grid)
    print()

Solving maze from file: tinySearch.lay
Solution:
% % % % % % % % % 
% . . . .   . . % 
% % % % . % %   % 
%       .       % 
% . % %   % % . % 
% . % .       . % 
% % % % % % % % % 
Path cost: 5
Number of nodes expanded: 24
Max tree depth: 5
Max fringe size: 23

Solving maze from file: smallSearch.lay
Solution:
% % % % % % % % % % % % % % % % % % % % 
% . . . . . . . . . . . . . . . .   . % 
% . % % . % % . % % . % % . % %   % . % 
%   % %   % . . . . .             % . % 
% % % % % % % % % % % % % % % % % % % % 
Path cost: 16
Number of nodes expanded: 36
Max tree depth: 15
Max fringe size: 17

Solving maze from file: trickySearch.lay
Solution:
% % % % % % % % % % % % % % % % % % % % 
% .                       . . % . . . % 
% . % % . % % . % % . % % . % % . % . % 
%                 . . . . . . . . % . % 
% % % % % % % % % % % % % % % % % % . % 
% . . . . . . . . . . . . . . . . . . % 
% % % % % % % % % % % % % % % % % % % % 
Path cost: 32
Number of nodes expanded: 59
Max tree depth: 31